In [6]:
import requests
import time
import re
import openpyxl
import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import date

In [7]:
GLOBAL = 'https://www.gipernn.ru'
LINK = 'https://www.gipernn.ru/biznes-centry?per-page=50'

Добыча колличества ссылок

In [8]:
html = requests.get(LINK)
soup = BeautifulSoup(html.text, 'html.parser')
count = soup.find('div', class_='count')

count = int(count.text.split(':')[1].strip())
page_count = int(np.ceil(count / 50))

print(count)

198


Составляем массив ссылок

In [9]:
links = []
for page in tqdm(range(page_count + 1), ascii=True):
    html = requests.get(f"{LINK}&page={page}")
    soup = BeautifulSoup(html.text, 'html.parser')
    trs = soup.find('tbody').find_all('tr')
    for tr in trs:
        link = tr.find('a')
        if link:
            link = link.get('href')
            link = GLOBAL + link
            links.append(link)
    links = list(set(links))
print(len(links))   


100%|##########| 5/5 [00:01<00:00,  4.22it/s]

198


Составление своей базы данных

In [10]:
df = pd.DataFrame()
for link in tqdm(links, ascii=True):
    html = requests.get(link)
    soup = BeautifulSoup(html.text, 'html.parser')

    tmp = {} # словарь для временных данных
    tmp['Ссылка'] = link

    header = soup.find('h1')
    if header:
        header = header.text.strip().replace('\xa0', ' ')
    tmp['Заголовок'] = header

    price = soup.find('div', class_ = 'price')
    if price:
        price = price.text.replace('\xa0', ' ').strip().split('  ')
    tmp['Цена'] = price

    description = soup.find('div', class_='formatted-text')
    if description:
        description = description.text.strip().replace('\xa0', ' ')
    tmp['Описание'] = description
    
    try:
        w0 = soup.find('table', id = 'w0')
        if w0:
            w0 = w0.find_all('tr')
            for tr in w0:
                key = tr.find('th').text.strip()
                val = tr.find('td').text.strip()
                tmp[key] = val
    except:
        print('w0 not found')

    df = pd.concat([df, pd.DataFrame(tmp, index=[0])], ignore_index=True)

df['Источник'] = 'gipernn'
df['Сегмент'] = 'Комерческая недвижимость'
df['Подсегмент'] = 'Аренда бизнес-центров'
df['Город'] = 'Нижний Новгород'
df['Дата парсинга'] = str(date.today())

100%|##########| 198/198 [00:52<00:00,  3.81it/s]


Посмотрим на результаты вкрадце

In [11]:
df.head()

,Ссылка,Заголовок,Цена,Описание,Источник,Сегмент,Подсегмент,Город,Дата парсинга
0,https://www.gipernn.ru/biznes-centry/geroya-po...,"Офисный центр Героя Попова, 47 в Нижнем Новгор...",None,None,gipernn,Комерческая недвижимость,Аренда бизнес-центров,Нижний Новгород,2022-12-22
1,https://www.gipernn.ru/biznes-centry/zaycev-bi...,Бизнес-центр Зайцев Бизнес Парк в Нижнем Новго...,None,None,gipernn,Комерческая недвижимость,Аренда бизнес-центров,Нижний Новгород,2022-12-22
2,https://www.gipernn.ru/biznes-centry/rozhdestv...,"Офисный центр Рождественская, 16д в Нижнем Нов...",None,None,gipernn,Комерческая недвижимость,Аренда бизнес-центров,Нижний Новгород,2022-12-22
3,https://www.gipernn.ru/biznes-centry/dialog-pr...,Бизнес-центр Диалог в Нижегородской областигор...,None,None,gipernn,Комерческая недвижимость,Аренда бизнес-центров,Нижний Новгород,2022-12-22
4,https://www.gipernn.ru/biznes-centry/na-alekse...,Бизнес-центр На Алексеевской в Нижнем Новгород...,None,None,gipernn,Комерческая недвижимость,Аренда бизнес-центров,Нижний Новгород,2022-12-22


Экспортируем таблицу

In [12]:
df.to_excel('business-rent.xlsx', index=False)